# Análise comparativa de modelos

In [1]:
# Bibliotecas
 
from IPython.display import display, Markdown, HTML
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



## 1. Obtenção de dados

In [2]:
#Dicionário de dados
df = pd.read_csv("../data/raw/penguins.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,species,"Espécie do pinguim (Adelie, Chinstrap, Gentoo)",qualitativa,nominal
1,island,"Ilha onde o pinguim foi encontrado (Biscoe, Dr...",qualitativa,nominal
2,bill_length_mm,Comprimento do bico em milímetros,quantitativa,contínua
3,bill_depth_mm,Profundidade do bico em milímetros,quantitativa,contínua
4,flipper_length_mm,Comprimento da nadadeira em milímetros,quantitativa,contínua
5,body_mass_g,Massa corporal em gramas,quantitativa,contínua
6,sex,Sexo do pinguim (Masculino ou Feminino),qualitativa,nominal
7,year,Ano em que os dados foram coletados,quantitativa,discreta


# 2. Preparação de dados 

- Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados.



In [3]:
target_column = 'species'
nominal_columns = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .to_list()
)
continuous_columns = (
    df_dict
    .query("subtipo == 'continua'")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

# 2. Pipeline de Pré-processamento de dados

In [4]:
#Pipeline para variáveis nominais e tratamento de dados discrepantes

# tratamento de dados discrepantes
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # codificação de variáveis
    ('normalization', StandardScaler()) # normalização de dados
])
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')), # tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns)
])

model = LogisticRegression()

# 3. Seleção de modelos

In [5]:
# experiment settings
n_splits_comparative_analysis = 10
n_folds_grid_search = 5
test_size = .2
random_state = 42
scoring = 'accuracy'
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# model settings
max_iter = 1000
models = [
    ('K-Nearest Neighbors', KNeighborsClassifier(), {"n_neighbors": range(3, 20, 2), 'weights': ['uniform', 'distance']}),
    ('Suport Vector Machines', SVC(random_state=random_state, max_iter=max_iter), {"kernel": ["linear", "rbf"], 'C':[1,10,100,1000],'gamma':[0.0001, 0.001, 0.1, 1]}),
    ('Decision Tree',  DecisionTreeClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8]}),
    ('Random Forest',  RandomForestClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8], 'n_estimators': [10, 30]}),
]


In [6]:
results = pd.DataFrame({})
cross_validate_grid_search = KFold(n_splits=n_folds_grid_search)
cross_validate_comparative_analysis = ShuffleSplit(n_splits=n_splits_comparative_analysis, test_size=test_size, random_state=random_state)
for model_name, model_object, model_parameters in models:
    print(f"running {model_name}...")
    model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cross_validate_grid_search
    )
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model_grid_search)
    ])
    scores = cross_validate(
        estimator=approach,
        X=X,
        y=y,
        cv=cross_validate_comparative_analysis,
        n_jobs=-1,
        scoring=metrics
    )
    scores['model_name'] = [model_name] * n_splits_comparative_analysis
    display(pd.DataFrame(scores).agg(['mean', 'std']))
    results = pd.concat([results, pd.DataFrame(scores)], ignore_index=True)

running K-Nearest Neighbors...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TypeError: Could not convert K-Nearest NeighborsK-Nearest NeighborsK-Nearest NeighborsK-Nearest NeighborsK-Nearest NeighborsK-Nearest NeighborsK-Nearest NeighborsK-Nearest NeighborsK-Nearest NeighborsK-Nearest Neighbors to numeric

In [ ]:
def highlight_best(s, props=''):
    if s.name[1] != 'std':
        if s.name[0].endswith('time'):
            return np.where(s == np.nanmin(s.values), props, '')
        return np.where(s == np.nanmax(s.values), props, '')

display(Markdown("### 3.1 Resultados gerais"))
(
    results
    .groupby('model_name')
    .agg(['mean', 'std']).T
    .style
    .apply(highlight_best, props='color:white;background-color:gray;font-weight: bold;', axis=1)
    .set_table_styles([{'selector': 'td', 'props': 'text-align: center;'}])
)